> (c) 2022 Tim C. Smith, All rights reserved
>


# Universal Bank Promotional Campaign: A practical illustration of *model performance assessment using confusion matrices*


## Problem Statement

Universal Bank has begun a program to encourage existing customers to borrow via a consumer loan program. 

The bank has tested a loan promotion on a random sample of 5000 customers. This test promotion resulted in 480 of the 5000 existing customers accepting the offer. 

The bank is intrigued by the success of this promotion. It has hired you to help them develop a model to identify which of its remaining customers may accept a similar promotion. 

They hired you to help them reduce the promotion costs and target the offer to only a subset of its customers that or more likely to accept the offer. They disclosed that the cost to promote this offer is \\$10 dollars per customer, and the profit from obtaining a loan customer is \\$100. They have an additional 50,000 customers that have not been contacted about the promotion. 

## Preliminary business problem scoping

Before starting your analysis, you jot down what the profit and loss (P&L) must have been from their marking trial...

In [1]:
TP_profit = 100 - 10   # the loan profit minus the targeting cost
FP_profit = -10   # no loan profit, just the targeting cost
TN_profit = 0 # they wouldn't take a loan, and we didn't spend the money targeting them
FN_profit = -90 # they would have taken the loan, but we didn't target them 

# in the null model there are only FP's and TP's because everyone is considered a possible customer
original_profit = 480*TP_profit + (5000-480)*FP_profit
print(f"${original_profit:,.2f}")

$-2,000.00


Using this approach on the remaining 50000 customers would not be a good business decision, as it would result in a loss of...

In [2]:
print(f"${original_profit*50000/5000:,.2f}")

$-20,000.00


Based on this initial analysis, it's clear that the bank cannot continue this promotion unless they have a model to help them identify the best potential customers. The best model possible would be 100% accurate. Universal Bank's initial test resulted in 480 or 5000 customers choosing to take the offer. The rate 480/5000 is, therefore, the expected rate of customers that take the request, and thus the expected profit result from a perfectly accurate model would be:

In [3]:
print(f"${480*50000/5000*TP_profit:,.2f}")

$432,000.00


> **Universal Bank is making the right decision in bringing you in to help them. You're probably now thinking you should renegotiate your constulting fee :)**

## Load, Explore and Clean Data

In [4]:
import pandas as pd

# Load the data direct from GitHub
bank_df = pd.read_csv('../Data/UniversalBank.csv') # change this path depending on where you store the file on your computer
bank_df.head(20)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
5,6,37,13,29,92121,4,0.4,2,155,0,0,0,1,0
6,7,53,27,72,91711,2,1.5,2,0,0,0,0,1,0
7,8,50,24,22,93943,1,0.3,3,0,0,0,0,0,1
8,9,35,10,81,90089,3,0.6,2,104,0,0,0,1,0
9,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0


Check column names, and for convenience, remove whitespaces...

In [5]:
bank_df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [6]:
bank_df.columns = [s.strip().upper().replace(' ', '_') for s in bank_df.columns] 
bank_df.columns

Index(['ID', 'AGE', 'EXPERIENCE', 'INCOME', 'ZIP_CODE', 'FAMILY', 'CCAVG',
       'EDUCATION', 'MORTGAGE', 'PERSONAL_LOAN', 'SECURITIES_ACCOUNT',
       'CD_ACCOUNT', 'ONLINE', 'CREDITCARD'],
      dtype='object')

We decide that a couple of variables aren't predictors; therefore we drop them and then check if there are any missing values in the remaining variables...

In [7]:
# drop ID, and Zip Code as predictors
bank_df = bank_df.drop(columns=['ID', 'ZIP_CODE'])

# check for missing values
bank_df.isnull().sum()

AGE                   0
EXPERIENCE            0
INCOME                0
FAMILY                0
CCAVG                 0
EDUCATION             0
MORTGAGE              0
PERSONAL_LOAN         0
SECURITIES_ACCOUNT    0
CD_ACCOUNT            0
ONLINE                0
CREDITCARD            0
dtype: int64

Check the variable types

In [8]:
bank_df.dtypes

AGE                     int64
EXPERIENCE              int64
INCOME                  int64
FAMILY                  int64
CCAVG                 float64
EDUCATION               int64
MORTGAGE                int64
PERSONAL_LOAN           int64
SECURITIES_ACCOUNT      int64
CD_ACCOUNT              int64
ONLINE                  int64
CREDITCARD              int64
dtype: object

In [9]:
bank_df.SECURITIES_ACCOUNT.unique()

array([1, 0])

In [10]:
bank_df.SECURITIES_ACCOUNT = bank_df.SECURITIES_ACCOUNT.astype('category')

In [11]:
bank_df.CD_ACCOUNT.unique()

array([0, 1])

In [12]:
bank_df.CD_ACCOUNT = bank_df.CD_ACCOUNT.astype('category')

In [13]:
bank_df.ONLINE.unique()

array([0, 1])

In [14]:
bank_df.ONLINE = bank_df.ONLINE.astype('category')

In [15]:
bank_df.CREDITCARD.unique()

array([0, 1])

In [16]:
bank_df.CREDITCARD = bank_df.CREDITCARD.astype('category')

In [17]:
bank_df.PERSONAL_LOAN.unique()

array([0, 1])

In [18]:
bank_df.PERSONAL_LOAN = bank_df.PERSONAL_LOAN.astype('category')

In [19]:
bank_df.EDUCATION.unique()

array([1, 2, 3])

In [20]:
bank_df.EDUCATION = bank_df.EDUCATION.astype('category')

In [21]:
bank_df.describe()

,AGE,EXPERIENCE,INCOME,FAMILY,CCAVG,MORTGAGE
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,45.338400,20.104600,73.774200,2.396400,1.937938,56.498800
std,11.463166,11.467954,46.033729,1.147663,1.747659,101.713802
min,23.000000,-3.000000,8.000000,1.000000,0.000000,0.000000
25%,35.000000,10.000000,39.000000,1.000000,0.700000,0.000000
50%,45.000000,20.000000,64.000000,2.000000,1.500000,0.000000
75%,55.000000,30.000000,98.000000,3.000000,2.500000,101.000000
max,67.000000,43.000000,224.000000,4.000000,10.000000,635.000000


## Split and Normalize Data

K-nn models are sensitive to differences in scale; therefore, we should begin by eliminating any differences in scale between the predictors/features. To accomplish this, we will standardize the values of each variable.

We will use the popular sklearn library's 'standard scaler' to accomplish this. This library contains many of the common functions we require when conducting analytics. The standard scaler function will standardize our variables. To achieve this, we will first need to train the scaler on the training data and then apply this trained scaler to standardize both the training and validation sets. 

In [22]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 

# split the data into validation and training set
train_df, validation_df = train_test_split(bank_df, test_size=0.3, random_state=1)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'PERSONAL_LOAN'
predictors = list(bank_df.columns)
predictors.remove(target)

# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
scaler.fit(train_df[predictors])

# Transform the predictors of training and validation sets
train_predictors = scaler.transform(train_df[predictors]) # train_predictors is not a numpy array
train_target = train_df[target] # train_target is now a series object

validation_predictors = scaler.transform(validation_df[predictors]) # validation_target is now a series object
validation_target = validation_df[target] # validation_target is now a series object

> * https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
> * https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

Let's now explore the results of the standardization (NOTE: The returned object from a scaler transform is a numpy array)

In [23]:
import numpy as np

print(train_predictors[:5], end="\n\n")
print(np.array(train_target[:5]), end="\n\n")
print(validation_predictors[:5], end="\n\n")
print(np.array(validation_target[:5]), end="\n\n")

[[ 0.13597665  0.15613728 -0.83705838 -0.34495524 -0.3657083  -1.05571315
  -0.55770661 -0.34173625 -0.25835623  0.83164805 -0.650648  ]
 [-0.64621189 -0.53857809 -0.75040637 -1.21794393  0.03787597  0.13545256
  -0.55770661 -0.34173625 -0.25835623  0.83164805 -0.650648  ]
 [ 1.1788947   1.28504976  1.24258976 -1.21794393  1.0756641  -1.05571315
  -0.55770661 -0.34173625 -0.25835623  0.83164805  1.53692933]
 [-1.42840043 -1.49381173 -0.18716833  1.40102213 -0.07743382  0.13545256
   1.86992266 -0.34173625 -0.25835623 -1.20243173 -0.650648  ]
 [-0.55930205 -0.62541751 -1.1403404   0.52803344 -0.99991216  0.13545256
  -0.55770661 -0.34173625 -0.25835623  0.83164805 -0.650648  ]]

[0 0 0 0 0]

[[-1.25458075 -1.32013289  0.22442869 -1.21794393  0.55677004  1.32661827
   0.48696992 -0.34173625 -0.25835623 -1.20243173  1.53692933]
 [-0.9069414  -0.9727752  -0.62042836  0.52803344 -0.59632789 -1.05571315
   0.44717272  2.92623333 -0.25835623 -1.20243173 -0.650648  ]
 [-0.99385124 -0.9727752  

## Train a K-NN model

You've heard that a good starting point in determining a k value is to try the square root of the total observations. Since there are 5000 observations, following this rule of thumb, we would select a k value of 70  (but it's best to choose an odd number so that we will use k=71).

> If k is an even number, there is a possibility that an observation could have the same number of nearest neighbors being one class (they took the loan) as the number being another (did not take the loan). In the cases of a tie, the SKLearn implementation of k-nn will select the first found. Though the chances of this happening are low, it's better to avoid this by choosing an odd number.

In [24]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=71,  metric='euclidean') # user euclidean distance

# We could choose other distance metrics; for a list of other metrics...
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics
# coverage of difference distance metrics is outside of this courses scope... but, you can experiment by changing the metric
# for example...
#knn = KNeighborsClassifier(n_neighbors=71,  metric='manhattan')

knn.fit(train_predictors, train_target)
knn_prediction_output = knn.predict(validation_predictors)
knn_prediction_output

array([0, 0, 0, ..., 0, 0, 0])

> * https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
> * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.DistanceMetric.html#sklearn.metrics.DistanceMetric


In [25]:
unique, counts = np.unique(knn_prediction_output, return_counts=True)
dict(zip(unique, counts))

{0: 1459, 1: 41}

> Note that we know that the TP rate is 480/5000 = 9.6% in the sample, but our predictions are identifying 41/1459=2.8%. This is a bit of a 'red flag' to indicate that the precision of our model may be poor - which is most likely due to the imbalance between classes in our training data (480 in one class and 4520 in the other). Such imbalances can be a problem with many machine learning algorithms, including k-nn. Addressing this problem is a topic for another class, but the influence of the unbalanced data can be partially mitigated by using a smaller k value. We will see this in practice when we cover hyper-parameter tuning in the next class.

> For now, we will ignore this potential opportunity to improve our model and move ahead with our analysis to see if this model is sufficient to help Universal Bank generate profit. 

## Measure performance of model using confusion matrix

In [26]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(validation_target, knn_prediction_output)

> * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

In [27]:
confusion

array([[1349,    2],
       [ 110,   39]])

In [28]:
TP = confusion[1, 1] # True Positives
TN = confusion[0, 0] # True Negatives
FP = confusion[0, 1] # False Positives
FN = confusion[1, 0] # False Negatives

### Accuracy:

How often was the model correct?

In [29]:
classification_accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"{classification_accuracy:.4f}")

0.9253


### Misclassification Rate:

How often was the model incorrect?

In [30]:
classification_error = (FP + FN) / (TP + TN + FP + FN)
# this is the same as ...
# classification_error = 1-classification_accuracy

print(f"{classification_error:.4f}")

0.0747


### Precision: 

When a positive value is predicted, how often is the prediction correct?

In other wordes: How "precise" is the classifier when predicting positive instances?

In [31]:
precision = TP / (TP + FP)
print(f"{precision:.4f}")

0.9512


### Recall (aka sensititivy):

Ability of a classification model to identify all relevant instances. 
Also referred to as Sensitivity, Probability of Detection, True Positive Rate


In [32]:
recall = TP / (TP + FN)
print(f"{recall:.4f}")

0.2617


### F1 Score

This is a measure that takes the harmonic mean of precision and recall.

In [33]:
f1_Score = (2 * precision * recall) / (precision + recall)
print(f"{f1_Score:.4f}")

0.4105


* As the analytics consultant, you know what all the above performance measures mean - but while presenting this information to the CEO, she became impatient and interrupted you to ask how profit and loss will be impacted if they choose to use this model over simply targeting everyone (you then take a note to yourself to place more focus on profit and loss when speaking with future CEO's)*

Let's look at how we could address this question...

## Is our k-nn model a better model that the null (promote to all) model?


Based on your initial business problem scoping, you calculated Universal Bank's expected profit from targeting the remaining 50,000 customers without the support of a model was \\$-20,000.00. 

Let's calculate how much our model improves things...

#### What would be the expected returns from not using a model?

From the initial trial, 480 of the 5000 (9.6%) customers targeted took the loan, and 5000-480 (90.4%) didn't take the loan. Since the initial test was on a random selection of customers and did not use a model, then the expected percentage of the 50000 customer will also be 9.6% and 90.4%.

Let's look at the confusion matrix of the performance of the model on the validation data

In [34]:
confusion 

array([[1349,    2],
       [ 110,   39]])

Let's convert these values into percentages

In [35]:
confusion_perc = confusion/(.3*5000)  # since the confusion matrix is for the performance on the validation data, it's .30% of 5000, or 1500. 
confusion_perc

array([[0.89933333, 0.00133333],
       [0.07333333, 0.026     ]])

Now, let's do a scaler multiplication of the number of customers that will be targetted in the newly proposed campaign (50,000 customers). This will give us the number of expected customers in each of the categories in the confusion matrix (TP, FP, TN, FN)

In [36]:
knn_result = confusion_perc * 50000
knn_result

array([[44966.66666667,    66.66666667],
       [ 3666.66666667,  1300.        ]])

Now, let's create a p_and_l matrix (profit and loss)

In [37]:
p_and_l = np.array([[TN_profit, FP_profit],[FN_profit, TP_profit]])
p_and_l

array([[   0,  -10],
       [-100,   90]])

Multiple our profit and loss matrix with our knn_result matrix to get profit and loss associated with each of the catgories (FP, TP, FN, TN)

In [38]:
knn_result * p_and_l

array([[      0.        ,    -666.66666667],
       [-366666.66666667,  117000.        ]])

To get the total profit, we simply need to sum up all the p_and_l's for each category (TP, FP, TN, FN)

In [39]:
model_profit = (knn_result * p_and_l).sum()

print(f"${model_profit:,.2f}")

$-250,333.33


As we can see, using a k value of 71 would result in a model that is expected to look over \\$250,000 in the upcoming campaign. This is much worse than the null model (where they simply don't use a model and target every customer).

## Testing other values of k

Clearly, our model is a failure. Despite some model measures seemingly looking ok (for instance, accuracy is 92.53%), using it would result in an expected loss of \\$250,333.33 (good thing we checked!). 

Let's spend a few minutes thinking about why that is.

Notice that though accuracy looks quite good, recall is relatively poor. Since the cost of an FN is very high (loss of \\$100) relative to the cost of a false positive (a loss of \\$$10), the distribution of the misclassified observations between FP and FN is very important. For any errors we must accept, it is much more advantageous if these are FP's. 

As we noted in the early stages of this evaluation (see note in "Train a K-NN model" section), our dataset is significantly unbalanced, with the number of customers not taking a loan nearly 10x higher than those that would take a loan. K-NN is particularly sensitive to this; in this case, the more significant proportion of observations where the customer did not take a loan makes it difficult for K-NN to identify the instances where they would take a loan. (since it's much more probably that a neighbor is a non-customer, and thus the voting will skew towards selecting an observation as a non-customer). This is particularly more problematic as the value of k increases. 

Is all lost?

There is still a chance that we can make the model better. Due to the unbalanced nature of this dataset, we'll most likely find that the model will perform better with a lower value of k. 

Let's test this hypothesis by trying a range of k values:

In [40]:
profits = []
for i in range(1,141,2):
    knn = KNeighborsClassifier(n_neighbors=i,  metric='euclidean')
    knn.fit(train_predictors, train_target)
    knn_prediction_output = knn.predict(validation_predictors)
    confusion = confusion_matrix(validation_target, knn_prediction_output)
    TP = confusion[1, 1] 
    TN = confusion[0, 0] 
    FP = confusion[0, 1] 
    FN = confusion[1, 0] 
    confusion_perc = confusion/1500 
    knn_result = confusion_perc * 50000
    p_and_l = np.array([[TN_profit, FP_profit],[FN_profit, TP_profit]])
    knn_result * p_and_l
    model_profit = (knn_result * p_and_l).sum()
    profits.append(model_profit)
    print(f"For k={i} model profit is ${model_profit:,.2f}")
print("*"*80)    
print(f"Max profit is {max(profits):,.2f}")
print("*"*80)    


For k=1 model profit is $122,666.67
For k=3 model profit is $76,666.67
For k=5 model profit is $39,333.33
For k=7 model profit is $1,666.67
For k=9 model profit is $-16,333.33
For k=11 model profit is $-54,333.33
For k=13 model profit is $-67,000.00
For k=15 model profit is $-92,000.00
For k=17 model profit is $-92,000.00
For k=19 model profit is $-99,000.00
For k=21 model profit is $-105,333.33
For k=23 model profit is $-118,000.00
For k=25 model profit is $-118,000.00
For k=27 model profit is $-130,666.67
For k=29 model profit is $-143,333.33
For k=31 model profit is $-149,666.67
For k=33 model profit is $-155,666.67
For k=35 model profit is $-168,333.33
For k=37 model profit is $-181,333.33
For k=39 model profit is $-187,333.33
For k=41 model profit is $-199,666.67
For k=43 model profit is $-193,333.33
For k=45 model profit is $-193,333.33
For k=47 model profit is $-199,666.67
For k=49 model profit is $-199,666.67
For k=51 model profit is $-206,000.00
For k=53 model profit is $-225,

## Business Impact Summary

Universal Bank's initial promotional campaign offered 5000 existing customers a loan product. Of the 5000 targeted, 480 accepted the offer. Since the cost of targeting a customer is \\$10 and the profit from getting a customer to take the loan is \\$100, this initial trial resulted in a loss of \\$2,000. If this 'null model' approach were used on the remaining 50,000 customers, this would result in a loss of \\$20,000. 

Using a k-nn model with k=1, the expected total profit is $122,666.67.

Based on the expected performance of the proposed model, Universal Bank could proceed with implementing a profitable campaign. But, the data shows a significant imbalance as there are far more customers that didn't take the offer than did. if we could reduce the negative effect of the unbalanced data set, there is an opportunity to develop a better performing model. (as a reminder, a 'perfect' model would produce a profit of $432,000.00 - though we can't expect a perfect model to be found, there is significant room for improvement).  
